# Point Regressor Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
dataDir = 'data/tower1.0/conmech/design_5_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir, loadDims=[0])

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.027434
std,0.005930
min,0.022934
25%,0.023780
50%,0.025250
75%,0.028404
max,0.065064


In [3]:
# plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

## 2. Filter and partition

In [3]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.025836
std,0.002804
min,0.022934
25%,0.023687
50%,0.024775
75%,0.027176
max,0.034477


## 3. Train and test random forest point regressors

In [4]:
rf = PointRegressor('Random Forest')
rf.trainModel(trainData, valData, 
              flatten=False, 
              logTrans=False, 
              ssTrans=True,
              useXFeatures=False,
              saveDir='results/rf03Tower/')

trainRes = rf.testModel(trainData)
testRes = rf.testModel(testData)

pd.set_option('display.float_format', lambda x: '%.3e' % x)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

trained 24 random forest models in 11.12 seconds


,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,8.567e-10,1.474e-05,7.797e-04,9.996e-01,1.000e+00,9.993e-01,9.981e-01
test,7.594e-09,4.166e-05,2.165e-03,9.951e-01,1.000e+00,9.931e-01,9.839e-01


## 4. Visualize some predictions

In [6]:
i = 8
pred = rf.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

UnboundLocalError: local variable 'height' referenced before assignment

In [ ]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = rf.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');